> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [ ]:
from typing import List
from pydantic import BaseModel


class HourlyUnits(BaseModel):
    time: str
    temperature_2m: str


class Hourly(BaseModel):
    time: List[str]
    temperature_2m: List[float]


class OpenMeteo(BaseModel):
    latitude: float
    longitude: float
    generationtime_ms: float
    utc_offset_seconds: int
    timezone: str
    timezone_abbreviation: str
    elevation: float
    hourly_units: HourlyUnits
    hourly: Hourly


#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, Enum
from sqlalchemy.ext.declarative import declarative_base
import enum
Base = declarative_base()
class SexoEnum(enum.Enum):
    F = "F"
    M = "M"

class DesignacaoEnum(enum.Enum):
    JuniorAnalyst = "Junior Analyst"
    Analyst = "Analyst"
    SeniorAnalyst = "Senior Analyst"
    Manager = "Manager"
    AVP = "AVP"
    VP = "VP"

class UnidadeEnum(enum.Enum):
    IT = "IT"
    Finance = "Finance"
    HR = "HR"
    Marketing = "Marketing"
    Legal = "Legal"
class Profissional(Base):
    __tablename__ = 'profissionais'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=False)
    sex = Column(Enum(SexoEnum), nullable=False)
    doj = Column(Date, nullable=False)  # Date of Joining
    current_date = Column(Date, nullable=False)
    designation = Column(Enum(DesignacaoEnum), nullable=False)
    age = Column(Integer, nullable=False)
    salary = Column(Float, nullable=False)
    unit = Column(Enum(UnidadeEnum), nullable=False)
    leaves_used = Column(Integer, nullable=False)
    leaves_remaining = Column(Integer, nullable=False)
    ratings = Column(Float, nullable=False)
    past_exp = Column(Float, nullable=False)
engine = create_engine('sqlite:///salaries.db')
Base.metadata.create_all(engine)


#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [ ]:
from sqlalchemy import create_engine

# Criação da engine e conexão com o banco SQLite
engine = create_engine('sqlite:///salarios.db')


#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Conexão com o banco
engine = create_engine('sqlite:///salarios.db')
Base = declarative_base()

# Modelo da tabela
class Profissional(Base):
    __tablename__ = 'profissionais'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(String)
    doj = Column(Date)
    current_date = Column(Date)
    designation = Column(String)
    age = Column(Integer)
    salary = Column(Float)
    unit = Column(String)
    leaves_used = Column(Integer)
    leaves_remaining = Column(Integer)
    ratings = Column(Float)
    past_exp = Column(Float)
# Criação da(s) tabela(s) no banco
Base.metadata.create_all(engine)


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# 1. Carregar os dados do CSV (ajuste o caminho se necessário)
df = pd.read_csv("salaries.csv", parse_dates=['DOJ', 'CURRENT DATE'])

# 2. Conectar ao banco de dados SQLite
engine = create_engine("sqlite:///salarios.db")

# 3. Popular a tabela com os dados usando to_sql
df.to_sql(
    name='profissionais',
    con=engine,
    if_exists='append',  # não apaga a tabela, apenas adiciona
    index=False           # não adiciona uma coluna de índice
)


#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [ ]:
from sqlalchemy import create_engine, text

engine = create_engine("sqlite:///salarios.db")

query = """
SELECT 
    DESIGNATION, 
    MIN(SALARY) / 12 AS salario_min_mensal,
    MAX(SALARY) / 12 AS salario_max_mensal,
    AVG(SALARY) / 12 AS salario_medio_mensal
FROM profissionais
GROUP BY DESIGNATION;
"""

with engine.connect() as conn:
    result = conn.execute(text(query))
    for row in result:
        print(row)


In [ ]:
import pandas as pd

with engine.connect() as conn:
    df_result = pd.read_sql_query(query, conn)

print(df_result)


In [ ]:
from sqlalchemy.orm import Session
from sqlalchemy import select, func, column, Table, MetaData

# Carregar metadados dinamicamente
metadata = MetaData()
profissionais = Table("profissionais", metadata, autoload_with=engine)

stmt = select(
    profissionais.c.DESIGNATION,
    (func.min(profissionais.c.SALARY) / 12).label("salario_min_mensal"),
    (func.max(profissionais.c.SALARY) / 12).label("salario_max_mensal"),
    (func.avg(profissionais.c.SALARY) / 12).label("salario_medio_mensal")
).group_by(profissionais.c.DESIGNATION)

with Session(engine) as session:
    results = session.execute(stmt).all()
    for row in results:
        print(row)
